# How often does LAPD tweet? 

In [1]:
# https://github.com/dnanhkhoa/nb_black
%load_ext lab_black

In [2]:
import altair as alt
import pandas as pd
import matplotlib as mpl
import numpy as np
import json
import pytz
from datetime import datetime
import altair_latimes as lat

alt.themes.register("latimes", lat.theme)
alt.themes.enable("latimes")

ThemeRegistry.enable('latimes')

In [3]:
pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_rows", None)

### Read dump downloaded from trumptweetarchive.com

In [4]:
%run '00-lapd-tweets-processing.ipynb'

In [5]:
src = pd.read_csv(
    "output/lapdhq.csv",
    dtype={"id": str},
    low_memory=False,
)

In [6]:
src["user"] = "lapdhq"

### How many tweets?

In [7]:
len(src)

12226

### Clean up dates

In [8]:
src["id"] = src["id"].astype(str)
src["year"] = src["year"].astype(str)
src["month"] = src["month"].astype(str)
src["day"] = src["day"].astype(str)
src["hour"] = src["hour"].astype(str)
src["date"] = pd.to_datetime(src["date"])

### Most popular tweets

In [12]:
src[["id", "date", "tweet", "likes_count"]].sort_values(
    "likes_count", ascending=False
).head(5)

,id,date,tweet,likes_count
892,1239000513850380288,2020-03-14,"During these times, we must work together and not against each other. Here are the facts: -There is no food shortage &amp; stores will restock, so please don’t hoard. -The water supply is clean &amp; safe to drink, so there’s no need to buy water in bulk. -We are in this together.",35861
1649,1112947396562960384,2019-04-01,Eric Holder is wanted for Homicide in the shooting of Nipsey Hussle. He was last seen in a 2016 white 4 door Chevy Cruze CA license plate 7RJD742. Anyone with information related to his whereabouts or this deadly shooting is urged to contact South Bureau Homicide at 323-786-5100 https://t.co/3pX4fbezDs,34286
3668,938574653085302785,2017-12-06,"For some it might just be a cat. But to others it’s a friend, a loved one, a companion. The #LAPD has a motto, “To Protect and to Serve”, and our officers have been displaying that in many ways during the #CreekFire &amp; #SkirballFire https://t.co/kmyHhvy9Fe",33594
2401,1037786719989329920,2018-09-06,What’s a day in Los Angeles without some @BTS_twt excitement! Always a great time meeting the #BTSArmy lining up outside Staples Center for tonight’s concert. Enjoy the show — we’ll be out here to make your day as safe and enjoyable as possible #BtsLoveYourselfTour https://t.co/vkWmcxMXuK,26360
867,1242926931298803712,2020-03-25,@theashleyray Hello. Can you please have your co-worker or their friend send us a Direct Message with more details about this alleged incident. We have no evidence of any LAPD officers being involved in these types of “spot checks”. Thanks,22594


In [21]:
src[["id", "date", "tweet", "retweets_count"]].sort_values(
    "retweets_count", ascending=False
).head(5)

,id,date,tweet,retweets_count
1649,1112947396562960384,2019-04-01,Eric Holder is wanted for Homicide in the shooting of Nipsey Hussle. He was last seen in a 2016 white 4 door Chevy Cruze CA license plate 7RJD742. Anyone with information related to his whereabouts or this deadly shooting is urged to contact South Bureau Homicide at 323-786-5100 https://t.co/3pX4fbezDs,34457
2643,1014724487277711360,2018-07-04,"Please Retweet — We need your help to locate the family of this child found today at Union Station. It’s believed he is 5-7 years old, deaf and autistic and we have been unable to communicate with him. Any info please contact LAPD Transit Services Division at (213) 922-1410 https://t.co/pgCPKd8KY1",13475
1975,1078165199113183232,2018-12-26,We want to see this face all over Twitter until he is caught — he is the suspect in the senseless killing of Newman Police Officer Ronil Singh. You will be caught... https://t.co/CfuNNC076z,13056
892,1239000513850380288,2020-03-14,"During these times, we must work together and not against each other. Here are the facts: -There is no food shortage &amp; stores will restock, so please don’t hoard. -The water supply is clean &amp; safe to drink, so there’s no need to buy water in bulk. -We are in this together.",11844
1843,1090306829588918272,2019-01-29,"Attention Los Angeles — this guy brutally punched two women at a hotdog stand on Jan. 26 in the area of 6th &amp; Spring. Someone knows him, and we would like to be one of those people. If you have any info contact Detective Gonzalez 213-996-1851 (after hours contact 213-486-6606). https://t.co/DN1Og1lToM",10754


### Tweets per month, year

In [23]:
month_year = src.groupby(["year", "month"]).agg({"date": "count"}).reset_index()

In [24]:
month_year.tail()

,year,month,date
110,2020,8,53
111,2020,9,67
112,2021,1,93
113,2021,2,80
114,2021,3,87


### What do they average each day in retweets, likes and replies

In [26]:
lapd_engagements_day = (
    src.groupby(["month", "year"])
    .agg(
        {
            "id": "size",
            "retweets_count": ["sum", "mean"],
            "likes_count": ["sum", "mean"],
        }
    )
    .round()
    .reset_index()
)

In [27]:
lapd_engagements_day.columns = (
    lapd_engagements_day.columns.map("_".join).str.strip().str.rstrip("_")
)

In [28]:
lapd_engagements_day.tail()

,month,year,id_size,retweets_count_sum,retweets_count_mean,likes_count_sum,likes_count_mean
110,9,2016,83,1611,19.0,3756,45.0
111,9,2017,142,3791,27.0,11185,79.0
112,9,2018,86,14081,164.0,39179,456.0
113,9,2019,55,10112,184.0,41040,746.0
114,9,2020,67,7397,110.0,32444,484.0


In [37]:
chart = (
    alt.Chart((lapd_engagements_day.query('year > "2016"')))
    .mark_bar(size=10, opacity=1)
    .encode(
        x=alt.X(
            "month:T",
            title=" ",
            axis=alt.Axis(values=["1", "12"], grid=False, tickCount=1, format="%b"),
        ),
        y=alt.Y(
            "id_size:Q",
            stack=None,
            title=" ",
            axis=alt.Axis(
                gridWidth=0.6,
                gridColor="#dddddd",
                offset=6,
                tickSize=0,
                domainOpacity=0,
                tickCount=2,
                format="2,f",
            ),
            scale=alt.Scale(domain=(0, 200)),
        ),
        facet=alt.Facet("year:N", columns=4, title=" "),
    )
    .properties(width=150, height=100, title="Monthly tweets by @LAPDHQ")
)

chart.configure_view(strokeOpacity=0).configure_legend(
    orient="top", symbolType="stroke"
).configure_legend(symbolType="square")

alt.Chart(...)